In [1]:
from pynvml import *

nvmlInit()
vram = nvmlDeviceGetMemoryInfo(nvmlDeviceGetHandleByIndex(1)).free/1024.**2
print('GPU1 Memory: %dMB' % vram)
if vram < 8000:
    raise Exception('GPU Memory too low')
nvmlShutdown()

GPU1 Memory: 11172MB


In [2]:
import os
import cv2
import h5py
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import *
from collections import Counter
import seaborn as sns
from tqdm import tqdm
import pandas as pd
import re
import time
import random

from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.regularizers import l2
from keras.utils.vis_utils import model_to_dot
import keras.backend as K
from make_parallel import make_parallel

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
IMAGE_DIR = 'image_contest_level_2'

MODEL_NAME = 'model_l2加层生成器2_0.997656.h5'

Using TensorFlow backend.


In [3]:
df = pd.read_csv('image_contest_level_2/labels.txt', sep=' ', header=None)
characters = u'0123456789()+-*/=君不见黄河之水天上来奔流到海复回烟锁池塘柳深圳铁板烧; '

labels_len = np.array(map(lambda x:len(x.decode('utf-8')), df[0]))
n_len = 45
n, width, height, n_class, channels = 100000, 900, 72, len(characters), 3

In [4]:
def decode(out):
    return ''.join([characters[x] for x in out if x < n_class-1 and x > -1])

def disp2(img):
    cv2.imwrite('a.png', img)
    return Image('a.png')

def disp(img, txt=None, first=False):
    global index
    if first:
        index = 1
        plt.figure(figsize=(16, 9))
    else:
        index += 1
    plt.subplot(4, 1, index)
    if len(img.shape) == 2:
        plt.imshow(img, cmap='gray')
    else:
        plt.imshow(img[:,:,::-1])
    if txt:
        plt.title(txt)

# 读取测试集

In [5]:
X = np.zeros((n, width, height, channels), dtype=np.uint8)

for i in tqdm(range(n)):
    img = cv2.imread('crop_split_test/%d.png'%i).transpose(1, 0, 2)
    a, b, _ = img.shape
    X[i, :a, :b] = img

100%|██████████| 100000/100000 [01:12<00:00, 1381.44it/s]


# 载入模型

In [6]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    y_pred = y_pred[:, 2:, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

rnn_size = 128

l2_rate = 1e-5

input_tensor = Input((width, height, 3))
x = input_tensor
for i, n_cnn in enumerate([2, 3, 6]):
    for j in range(n_cnn):
        x = Conv2D(32*2**i, (3, 3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_rate))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
    x = MaxPooling2D((2, 2))(x)

cnn_model = Model(input_tensor, x, name='cnn')

input_tensor = Input((width, height, 3))
x = cnn_model(input_tensor)

conv_shape = x.get_shape().as_list()
rnn_length = conv_shape[1]
rnn_dimen = conv_shape[3]*conv_shape[2]

print conv_shape, rnn_length, rnn_dimen

x = Reshape(target_shape=(rnn_length, rnn_dimen))(x)
rnn_length -= 2

x = Dense(rnn_size, kernel_initializer='he_normal', kernel_regularizer=l2(l2_rate))(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.25)(x)

gru_1 = GRU(rnn_size, implementation=2, return_sequences=True, name='gru1')(x)
gru_1b = GRU(rnn_size, implementation=2, return_sequences=True, go_backwards=True, name='gru1_b')(x)
gru1_merged = add([gru_1, gru_1b])

gru_2 = GRU(rnn_size, implementation=2, return_sequences=True, name='gru2')(gru1_merged)
gru_2b = GRU(rnn_size, implementation=2, return_sequences=True, go_backwards=True, name='gru2_b')(gru1_merged)
x = concatenate([gru_2, gru_2b])
x = Dropout(0.25)(x)
x = Dense(n_class, activation='softmax', kernel_regularizer=l2(l2_rate))(x)
rnn_out = x
base_model = Model(input_tensor, x)

base_model2 = make_parallel(base_model, 4)

[None, 112, 9, 128] 112 1152


# 预测


In [8]:
def disp3(index):
    s = decode(out[index])
    plt.figure(figsize=(16, 4))
    plt.imshow(X[index].transpose(1, 0, 2))
    plt.title('pred:%s'%s)

In [11]:
z = '0.997656'
base_model.load_weights('model_l2加层生成器2_%s.h5' % z)
y_pred = base_model2.predict(X, batch_size=500, verbose=1)
out = K.get_value(K.ctc_decode(y_pred[:,2:], input_length=np.ones(y_pred.shape[0])*rnn_length)[0][0])[:, :n_len]

100000/100000 [==============================] - 197s   


In [12]:
ss = map(decode, out)

vals = []
errs = []
for i in tqdm(range(100000)):
    a = ss[i].split(';')
    s = a[-1]
    for x in a[:-1]:
        x, c = x.split('=')
        s = s.replace(x, c+'.0')
    
    val = ''
    try:
        val = '%.2f' % eval(s)
    except:
        ss[i] = ''
        errs.append(s)
    
    vals.append(val)
    
with open('result_%s.txt' % z, 'w') as f:
    f.write('\n'.join(map(' '.join, list(zip(ss, vals)))).encode('utf-8'))
    
print len(errs)
print 1-len(errs)/100000.

100%|██████████| 100000/100000 [00:01<00:00, 79744.05it/s]


42
0.99958
